In [1]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0,api_key="", model_name="gpt-4-turbo")

# llm_transformer = LLMGraphTransformer(llm=llm)
# from langchain_core.documents import Document


In [2]:
import pandas as pd

path = r"C:\Users\Area-51m\Downloads\data.csv"
path_2 = r"C:\Users\Area-51m\Downloads\combineddata.csv"
df = pd.read_csv(path)
df_2 = pd.read_csv(path_2, encoding='ISO-8859-1')  # Ignore invalid bytes


In [3]:
df_2.head()

,prompt,testcase_type,testcase_name,response,documentation
0,Validate a robot framework test case for testi...,keyword-driven,common,*** Settings ***\nDescription The test case 'c...,The test case 'common' typically covers functi...
1,Build a robot framework test case for testing ...,keyword-driven,customer,*** Settings ***\nDescription This test case w...,The test case 'customer' generally covers the ...
2,Validate a robot framework test case for testi...,keyword-driven,invoice,*** Settings ***\nDescription This test case v...,The 'invoice' test case evaluates the function...
3,Create a robot framework test case for testing...,keyword-driven,processpayment,*** Settings ***\nDescription The 'processpaym...,The 'processpayment' test case primarily verif...
4,Build a robot framework test case for testing ...,keyword-driven,token,*** Settings ***\nDescription 'Token' test cas...,The 'token' test case generally involves check...


In [4]:
df_2.isna().sum()

prompt           0
testcase_type    0
testcase_name    0
response         0
documentation    1
dtype: int64

In [5]:
df_2.isna().sum()

prompt           0
testcase_type    0
testcase_name    0
response         0
documentation    1
dtype: int64

In [6]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   prompt         1067 non-null   object
 1   testcase_type  1067 non-null   object
 2   testcase_name  1067 non-null   object
 3   response       1067 non-null   object
 4   documentation  1066 non-null   object
dtypes: object(5)
memory usage: 41.8+ KB


In [7]:
#drop null rows
df_2["documentation"] = df_2["documentation"].replace({pd.NA:"Unknown"})
df_2["prompt"] = df_2["prompt"].replace({pd.NA:"Unknown"})
df_2["testcase_type"] = df_2["testcase_type"].replace({pd.NA:"Unknown"})
df_2["testcase_name"] = df_2["testcase_name"].replace({pd.NA:"Unknown"})
df_2["response"] = df_2["response"].replace({pd.NA:"Unknown"})

In [8]:
import os
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase
from dotenv import load_dotenv
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
print(NEO4J_USERNAME, NEO4J_PASSWORD, NEO4J_URI)
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

with GraphDatabase.driver(NEO4J_URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connected successfully")

neo4j 4jUvURDEH_lhyi_A4mzJDSOPkkMHtIbCLGU7rvjpNRw neo4j+s://42e907ca.databases.neo4j.io
Connected successfully


In [9]:

def insert_data(tx, row):
    tx.run('''
    MERGE (tc:TestCase {
        prompt: $prompt,
        testcase_name: $testcase_name,
        response: $response,
        documentation: $documentation
    })

    MERGE (tt:TestcaseType {name: $testcase_type})
    MERGE (tc)-[:HAS_TYPE]->(tt)

    WITH tc, $response AS resp

    FOREACH (lib IN [lib IN split(resp, '\n') WHERE lib CONTAINS 'Library' | trim(split(lib, ' ')[1])] |
        MERGE (l:Library {name: lib})
        MERGE (tc)-[:USES_LIBRARY]->(l)
    )

    FOREACH (varLine IN [line IN split(resp, '\n') WHERE line CONTAINS '${' | 
        trim(split(line, '}')[0] + '}')
    ] |
        MERGE (v:Variable {name: varLine})
        MERGE (tc)-[:USES_VARIABLE]->(v)
    )

    FOREACH (validatedCase IN [line IN split(resp, '\n') WHERE toLower(line) CONTAINS 'validate' | 
        trim(split(line, ' ')[1])
    ] |
        MERGE (vtc:ValidatedCase {name: validatedCase})
        MERGE (tc)-[:VALIDATES]->(vtc)
    )
    ''', row)
    
def process_dataframe(driver, df):
    with driver.session() as session:
        for index, row in df.iterrows():
            session.execute_write(insert_data, row.to_dict())
            print(f"Processed row {index}")

In [10]:
# Create a driver instance
driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH)

# Process the DataFrame
process_dataframe(driver, df_2)

# Close the driver connection
driver.close()

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [13]:
from langchain_community.graphs import Neo4jGraph
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database="neo4j"
)

In [24]:
cypher = """
  MATCH (n) 
  RETURN count(n) AS numberOfNodes
  """

In [25]:
result = kg.query(cypher)
result

[{'numberOfNodes': 7543}]

In [14]:
cypher = """
  MATCH (n)
  DETACH DELETE n
  """
kg.query(cypher)

[]

In [35]:
cypher = """
  MATCH (tc:TestCase {testcase_name:"payment_processing"}) 
  RETURN tc
  """
kg.query(cypher)

[{'tc': {'response': '*** Settings ***\nDescription This test case validates the proper handling and processing of payments within the Proware system. It involves ensuring payments are correctly recorded, processed, and handled without errors.\nLibrary SeleniumLibrary\n*** Variables ***\n${USER_ACCOUNT, PAYMENT_TYPE} user_account_value, payment_type_value\n${CARD_INFORMATION} card_information_value\n${CONFIRMATION_MESSAGE, TRANSACTION_DETAILS} confirmation_message_value, transaction_details_value\n${PAYMENT_AMOUNT} payment_amount_value\n${PAYMENT_DATE, EXPECTED_STATUS} payment_date_value, expected_status_value\n${TRANSACTION_ID} transaction_id_value\n${PAYMENT_METHOD} payment_method_value\n${USER_ID, PAYMENT_ID} user_id_value, payment_id_value\n*** Test Cases ***\nValidate Payment_Processing\n[Description] This test case validates the proper handling and processing of payments within the Proware system. It involves ensuring payments are correctly recorded, processed, and handled withou

In [36]:
cypher = """
  MATCH (tc:TestCase) 
  RETURN count(tc) AS numberOfTestCases
  """
kg.query(cypher)

[{'numberOfTestCases': 1067}]

In [37]:
cypher = """
  MATCH (tc:TestCase {testcase_name:"payment_processing"}) 
  RETURN tc.testcase_name, tc.prompt
  """
kg.query(cypher)

[{'tc.testcase_name': 'payment_processing',
  'tc.prompt': 'Create a robot framework test case for testing Payment functionality in the Proware platform, specifically payment_processing.'},
 {'tc.testcase_name': 'payment_processing',
  'tc.prompt': 'Generate a robot framework test case for testing PaymentGateway functionality in the Web platform, specifically payment processing.'},
 {'tc.testcase_name': 'payment_processing',
  'tc.prompt': 'Create a robot framework test case for testing Payment functionality in the BlueCorp platform, specifically payment_processing.'},
 {'tc.testcase_name': 'payment_processing',
  'tc.prompt': 'Develop a robot framework test case for testing Payment Processing functionality in the SterlingCooper platform, specifically payment_processing.'},
 {'tc.testcase_name': 'payment_processing',
  'tc.prompt': 'Create a robot framework test case for testing the PaymentGateway functionality in the QuantumSoft platform, specifically payment_processing.'}]

Pattern matching with multiple nodes

In [41]:
cypher = """
  MATCH (tc:TestCase)-[:HAS_TYPE]->(tt:TestcaseType) 
  RETURN tc.testcase_name, tt.name LIMIT 5
  """
kg.query(cypher)

[{'tc.testcase_name': 'common', 'tt.name': 'keyword-driven'},
 {'tc.testcase_name': 'customer', 'tt.name': 'keyword-driven'},
 {'tc.testcase_name': 'invoice', 'tt.name': 'keyword-driven'},
 {'tc.testcase_name': 'processpayment', 'tt.name': 'keyword-driven'},
 {'tc.testcase_name': 'token', 'tt.name': 'keyword-driven'}]

In [49]:
cypher = """
  MATCH (tc:TestCase {testcase_name:"user_login"})-[:USES_LIBRARY]->(AppiumLibrary:Library) 
  RETURN tc.testcase_name, AppiumLibrary
  """
kg.query(cypher)

[{'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': 'SeleniumLibrary'}},
 {'tc.testcase_name': 'user_login',
  'AppiumLibrary': {'name': '

In [11]:
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings


vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(api_key=""),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name='testcases',
    node_label="TestCase",
    text_node_properties=['prompt', 'testcase_name', 'response', 'documentation'],
    embedding_node_property='embedding',
)

In [12]:
response = vector_index.similarity_search(
    "TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards"
)
print(response[0].page_content)


prompt: Generate a robot framework test case for testing Omnicorp Testcases functionality in the PayNOW Test Cases platform, specifically TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards.
testcase_name: TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards
response: *** Settings ***
Description This test case verifies that the system prevents a customer from making a payment using a debit card, ensuring that potentially erroneous or unsupported payment methods are not able to be processed.
Test Setup Open PayNOW Application ${URL}
Test Teardown Close All Browsers
Test Template Validate TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards
Resource ../../../../Config/super.resource
Library DataDriver ${TESTDATA_FOLDER} sheet_name=TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards
*** Test Cases ***
${test_case_name}
*** Keywords ***
Validate TC_04 Validate Customer Is Unable To Process Payment Through D

In [18]:
response = vector_index.similarity_search(
    "Validate a robot framework test case for testing API functionality in the ClientCentral_Keywords platform, specifically invoice."
)
print(response[0].page_content)


prompt: Validate a robot framework test case for testing API functionality in the ClientCentral_Keywords platform, specifically invoice.
testcase_name: invoice
response: *** Settings ***
Description This test case verifies the accuracy and completeness of the invoice generation system by checking details such as invoice number, date, customer details, item details, price, applicable tax, discounts, total amount, and payment status.
Library SeleniumLibrary
*** Variables ***
${UNIQUE_VALUE|INVOICE_NUMBER|DUE_AMOUNT|INVOICE_DUE_DATE|INVOICE_AMOUNT} unique_value|invoice_number|due_amount|invoice_due_date|invoice_amount_value
${MERCHANT|UNIQUE_VALUE|INVOICE_NUMBER|DUE_AMOUNT|INVOICE_DUE_DATE} merchant|unique_value|invoice_number|due_amount|invoice_due_date_value
${MERCHANT|UNIQUE_VALUE|INVOICE_NUMBER|DUE_AMOUNT|INVOICE_DUE_DATE|INVOICE_AMOUNT} merchant|unique_value|invoice_number|due_amount|invoice_due_date|invoice_amount_value
${UNIQUE_VALUE|INVOICE_NUMBER|DUE_AMOUNT|INVOICE_DUE_DATE} uni

In [20]:
from langchain.chains import RetrievalQA
from langchain_community.llms.ollama import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = Ollama(
            model="sta_llama3.1_v2",
            num_ctx=6096,
            temperature=0.1,
            callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        )
vector_qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_index.as_retriever()
)

response = vector_qa.run(
    "Validate a robot framework test case for testing API functionality in the ClientCentral_Keywords platform, specifically invoice."
)



# The RecommendationService is currently being updated to include a new feature 
# that will provide more personalized and accurate product recommendations to 
# users. This update involves leveraging user behavior and preference data to 
# enhance the recommendation algorithm. The status of this update is currently
# in progress.

C:\Users\Area-51m\AppData\Local\Temp\ipykernel_30192\929167872.py:5: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(


*** Settings ***
Description This test case verifies the accuracy and completeness of the invoice generation system by checking details such as invoice number, date, customer details, item details, price, applicable tax, discounts, total amount, and payment status.
Library SeleniumLibrary
*** Variables ***
${UNIQUE_VALUE|INVOICE_NUMBER|DUE_AMOUNT|INVOICE_DUE_DATE|INVOICE_AMOUNT} unique_value|invoice_number|due_amount|invoice_due_date|invoice_amount_value
${MERCHANT|UNIQUE_VALUE|INVOICE_NUMBER|DUE_AMOUNT|INVOICE_DUE_DATE} merchant|unique_value|invoice_number|due_amount|invoice_due_date_value
${MERCHANT|UNIQUE_VALUE|INVOICE_NUMBER|DUE_AMOUNT|INVOICE_DUE_DATE|INVOICE_AMOUNT} merchant|unique_value|invoice_number|due_amount|invoice_due_date|invoice_amount_value
${UNIQUE_VALUE|INVOICE_NUMBER|DUE_AMOUNT|INVOICE_DUE_DATE} unique_value|invoice_number|due_amount|invoice_due_date_value
${|MERCHANT|ACCOUNT_NUMBER|INVOICE_DUE_DATE} |merchant|account_number|invoice_due_date_value
*** Test Cases ***


KeyboardInterrupt: 

In [77]:
print(response)

Here is a Robot Framework test case for testing the User Access Control functionality in the InstaBank platform, specifically focusing on the manage_users functionality:

```robot
*** Settings ***
Documentation     This test case ensures the proper functionality of user access control features in the InstaBank platform, focusing on managing users. This includes user creation, role assignment, deactivation, and ensuring proper access rights are maintained.
Library           SeleniumLibrary

*** Variables ***
${USERNAME}       admin_user
${PASSWORD}       admin_password
${USER_ID}        test_user_01
${USER_ROLE}      Regular
${USER_STATUS}    Active
${NEW_USER_DETAILS}  name=John Doe|email=john.doe@example.com|password=securePass123
${EXPECTED_USERLIST}  John Doe, Jane Smith
${SEARCH_KEYWORD}     John Doe
${ACCESS_LEVEL}       High

*** Test Cases ***
Validate Manage Users
    [Documentation]    This test case validates the manage users functionality including user creation, role assign

In [63]:
from langchain.chains.graph_qa.cypher import GraphCypherQAChain

kg.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = llm,
    qa_llm = llm, graph=kg, verbose=True,
)

In [73]:
response_2 = cypher_chain.run(
    "TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards.",
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (tc:TestCase {testcase_name: "TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards"})-[:VALIDATES]->(vc:ValidatedCase)
RETURN tc, vc

Full Context:
[{'tc': {'response': '*** Settings ***\nDescription This test case verifies that the system prevents a customer from making a payment using a debit card, ensuring that potentially erroneous or unsupported payment methods are not able to be processed.\nTest Setup Open PayNOW Application ${URL}\nTest Teardown Close All Browsers\nTest Template Validate TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards\nResource ../../../../Config/super.resource\nLibrary DataDriver ${TESTDATA_FOLDER} sheet_name=TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards\n*** Test Cases ***\n${test_case_name}\n*** Keywords ***\nValidate TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards\n[Arguments] ${test_case

In [72]:
print(response_2)

{'query': 'TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards.', 'result': 'The test case "TC_04 Validate Customer Is Unable To Process Payment Through Debit Cards" ensures that the system prevents a customer from making a payment using a debit card. This is crucial as it ensures that only supported and correct payment methods are processed, maintaining the security and integrity of the payment system.'}
